# Importamos Librerias

In [ ]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim

# Dataset: IMDB Movie reviews sentiment classification

In [ ]:
num_words=30000
INDEX_FROM=3  # idx 0 => PAD, idx 1 => START, idx 2 => OOV (out of vocab.)
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_words+2,)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
num_words=len(np.unique(np.hstack(data)))
print("Categories:", np.unique(targets))
print("Number of unique words:", num_words)

Categories: [0 1]
Number of unique words: 30000


Agregar el siguiente archivo al Google Drive

https://drive.google.com/open?id=1K5r423yMxBb1Yz2uDT7lto60lu1jqEjl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 234.75892
Standard Deviation: 173


In [ ]:
print("Label:", targets[0])
print(data[0])

Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


# Traemos el vocabulario y armamos indice reverso

In [ ]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - INDEX_FROM, "#") for i in data[1]] )
print(decoded)

1654784/1641221 [==============================] - 0s 0us/step
# big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal measures the hair is big lots of boobs bounce men wear those cut tee shirts that show off their stomachs sickening that men actually wore them and the music is just # trash that plays over and over again in almost every scene there is trashy music boobs and paramedics taking away bodies and the gym still doesn't close for # all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh a

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/DeepLearning/Laboratorios/NLP/GoogleNews-vectors-negative300.bin", binary=True)  

In [ ]:
w2v.wv["car"]

# Armamos la matriz de embeddings

In [ ]:
embed_dim=300
embedding_matrix=np.zeros([num_words+4,embed_dim])
for word, idx in index.items():
  if idx <= num_words and word in w2v.wv:
    embedding_matrix[idx+INDEX_FROM,:]=w2v.wv[word]
    
embedding_matrix.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


(30004, 300)

# Hacemos que todos los reviews tengan el mismo largo

In [ ]:
maxlen=1000

In [ ]:
data = pad_sequences(data, maxlen=maxlen, value=0.0)

In [ ]:
len(data[0])

1000

In [ ]:
len(data[1])

1000

In [ ]:
data=np.array(data)

In [ ]:
data.shape

(50000, 1000)

# Armamos el modelo con una Conv1D

In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

In [ ]:
nb_words=num_words+4
num_filters=64

model = Sequential()
model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters*2, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

model.fit(data,targets,batch_size=32,epochs=5,validation_split=0.2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 300)         9001200   
                                                                 
 conv1d_2 (Conv1D)           (None, 1000, 64)          134464    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 500, 64)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 500, 128)          57472     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 128)              